<a href="https://colab.research.google.com/github/analyticsindiamagazine/Notebooks/blob/master/Sample_Fitness_Value_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Fitnes Value Calculation 

> The notebook has the starter code to kickstart with the Auto Feature Engineering. Below are the Descriptions for the Features and How to calculate them using Pyhton.

*Three Different Features needs to be created to Calculate the following.*

#### 1.   RECENCY - *How recently did an event happen prior to the anchor date?*

```
Details

1 recency feature per event, hence #recency features will be equal to  events, 
If a patient does not have the event, the value of the feature should be “999999999” 

Example – For patient_01, recency of event_1 = 29, recency of event_2 = 33
```
**Total number Features for Recency will be equal to 755**

####2.   FREQUENCY - *How many times did an event happen in a specific time frame?*
```
Details

Data has 3 years of patient history i.e. 36 months resulting in 1 frequency feature per event per month, total of
36 features per event. Hence #frequency features will be equal to 36 times #events
Example – For patient_01, frequency of event_1 in 1 month = 1, frequency of event_1 in 6 months = 3
```
**Total numner of Features for Frequency will be equal to 27,180**

#### 3.   NORM CHANGE - *Has the frequency of an event increased or decreased in a recent time frame (not more than 1.5 years) as compared to the previous time frame?*
```
Details

Data has 36 months of patient history and can be split into two time-periods using 18 split-points i.e. total of 18
features per event (split points 1 months to 18 months).
Frequency in time period x (days: 30,60,90….) = total events / days in the time period
Change in frequency = Frequency in time period 1 – Frequency in time period (1080 – x)
Example – For patient_01, frequency of event_1 in last 1 month is higher as compared to the previous
time period
```
**If change in frequency >0 then 1 else 0 - Total # Features will be equal to 13,590 (# normChange features will be equal to 18 times # events )**

# Load Dataset and Fitness Evaluation File

In [0]:
import pandas as pd
import numpy as np

In [0]:
import os
# Data Folder Path - os.chdir(r'C:\Users\nr10863\Desktop\ZS\DS Recruitment\MLDS Case\Data\DS_ML_Recruitment')
path = "" 

In [0]:
train = pd.read_csv("train_data.csv")
train_labels = pd.read_csv("train_labels.csv")

train_data = pd.merge(train, train_labels, on='patient_id', how='left')

del train, train_labels

In [0]:
train

In [0]:
## Read Fitness Score CSV...
allFeatures = pd.read_csv('fitness_values.csv')

In [0]:
## Format for Fitness CSV..
allFeatures.head()

,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
0,recency__event_name__event_1,415.562260,402.517278,315.016268,307.536555,1.007895
1,recency__event_name__event_2,334.024691,308.916367,312.396980,298.470307,1.033075
2,recency__event_name__event_3,430.727273,224.280543,277.010141,274.041707,1.899904
3,recency__event_name__event_4,334.842105,326.087912,223.508206,305.235029,1.402317
4,recency__event_name__event_5,478.988636,455.197872,306.338123,315.533077,1.083849


In [0]:
time_var = 'event_time'
id_var = 'patient_id'
y_var = 'outcome_flag'

In [0]:
def fitness_calculation(data):
    if ((data['sd_0'] == 0 ) and (data['sd_1'] == 0)) and (((data['avg_0'] == 0) and (data['avg_1'] != 0)) or ((data['avg_0'] != 0) and (data['avg_1'] == 0))):
        return 9999999999
    elif (((data['sd_0'] == 0 ) and (data['sd_1'] != 0)) or ((data['sd_0'] != 0) and (data['sd_1'] == 0))) and (data['avg_0'] == data['avg_1']):
        return 1
    elif ((data['sd_0'] != 0 ) and (data['sd_1'] != 0)) and (data['avg_0'] != 0):
        return ((data['avg_1']/data['sd_1'])/(data['avg_0']/data['sd_0']))
    elif ((data['sd_0'] != 0 ) and (data['sd_1'] != 0)) and ((data['avg_0'] == 0) and (data['avg_1'] != 0)):
        return 9999999999
    else:
        return 1

## Recency

> The below cell shows, how to calculate Recency as Feature.
> Fitness values must be validated from the fitness values .csv file for every Feature.




In [0]:
feature_name = 'recency__event_name__event_10'

In [0]:
column = feature_name.split('__')[1]
value = feature_name.split('__')[2]

patient_level_feature = pd.DataFrame(train_data[train_data[column]==value][['patient_id', 'outcome_flag', 'event_time']].groupby(['patient_id', 'outcome_flag'])['event_time'].min(). reset_index())
patient_level_feature.columns = ['patient_id', 'outcome_flag', 'feature_value']

## calculate the stats for Fitness scores..
avg1 = patient_level_feature[(patient_level_feature['outcome_flag']==1) & (patient_level_feature['feature_value']!=9999999999)]['feature_value'].mean()
sd1 = patient_level_feature[(patient_level_feature['outcome_flag']==1) & (patient_level_feature['feature_value']!=9999999999)]['feature_value'].std()
avg0 = patient_level_feature[(patient_level_feature['outcome_flag']==0) & (patient_level_feature['feature_value']!=9999999999)]['feature_value'].mean()
sd0 = patient_level_feature[(patient_level_feature['outcome_flag']==0) & (patient_level_feature['feature_value']!=9999999999)]['feature_value'].std()

## Record all the above stats for using the below naming convention.
fitness = pd.DataFrame([feature_name, avg1, avg0, sd1, sd0]).transpose()
fitness.columns = ['feature_name', 'avg_1', 'avg_0', 'sd_1', 'sd_0']
fitness['coefficient_of_variance'] = fitness.apply(fitness_calculation, axis=1)
fitness ## calculated Fitness Score..

,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
0,recency__event_name__event_10,414.603,435.795,313.627,324.751,0.985116


In [0]:
## Validate the Feature scores with the newly created Fitness Dataframe.
allFeatures[allFeatures.feature_name==feature_name] ## Recency score from Fitness File.

,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
9,recency__event_name__event_10,414.602778,435.794798,313.626875,324.751066,0.985116


## Frequency

> The below cell shows, how to calculate Frequency as Feature.
> Fitness values must be validated from the fitness values .csv file for every Feature.

In [0]:
feature_name = 'frequency__event_name__event_10__60'

In [0]:
event = feature_name.split('__')[1]
value = feature_name.split('__')[2]
time = feature_name.split('__')[3]

_data = train_data[(train_data[time_var]<=int(time))].reset_index(drop=True)
_freq = _data[[id_var, event, time_var]].groupby([id_var, event]).agg({time_var: len}).reset_index()
_freq.columns = [id_var, 'feature_name', 'feature_value']
_freq['feature_name'] = 'frequency__' + event + '__' + _freq['feature_name'].astype(str) + '__' + str(time)
_freq = _freq.reset_index(drop=True)
_df1 = pd.DataFrame(_freq['feature_name'].unique().tolist(), columns=['feature_name'])
_df2 = pd.DataFrame(_freq[id_var].unique().tolist(), columns=[id_var])
_df1['key'] = 1
_df2['key'] = 1
_freqTotal = pd.merge(_df2, _df1, on='key')
_freqTotal.drop(['key'], axis=1, inplace=True)
_freqTotal = pd.merge(_freqTotal, _freq, on=[id_var, 'feature_name'], how='left')
_freqTotal.fillna(0, inplace=True)
_df3 = train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
_freqTotal = _freqTotal.merge(_df3, on=id_var, how='left')
freqTotal = _freqTotal.copy()

_avg1 = freqTotal.loc[freqTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
_avg1.columns = ['feature_name', 'avg_1']
_sd1 = freqTotal.loc[freqTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
_sd1.columns = ['feature_name', 'sd_1']
_avg0 = freqTotal.loc[freqTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
_avg0.columns = ['feature_name', 'avg_0']
_sd0 = freqTotal.loc[freqTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
_sd0.columns = ['feature_name', 'sd_0']

_fitness_value = pd.merge(_avg1, _avg0, on='feature_name', how='left')
_fitness_value = pd.merge(_fitness_value, _sd1, on='feature_name', how='left')
_fitness_value = pd.merge(_fitness_value, _sd0, on='feature_name', how='left')

fitness = _fitness_value[_fitness_value.feature_name==feature_name]
fitness['coefficient_of_variance'] = fitness.apply(fitness_calculation, axis=1)
fitness ## Calculated Fitness Scores for Frequency..

C:\Users\nr10863\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
10,frequency__event_name__event_10__60,0.070782,0.062675,0.680199,0.718469,1.192886


In [0]:
allFeatures[allFeatures.feature_name==feature_name] ##Frequency Score from Fitness File.

,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
1284,frequency__event_name__event_10__60,0.070782,0.062675,0.680199,0.718469,1.192886


## NormChange 

> The below cell shows, how to calculate NormChange as Feature.
> Fitness values must be validated from the fitness values .csv file for every Feature.

In [0]:
feature_name = 'normChange__event_name__event_10__60'

In [0]:
column = feature_name.split('__')[1]
value = feature_name.split('__')[2]
time = feature_name.split('__')[3]


_data_post = train_data[train_data[time_var]<=int(time)].reset_index(drop=True)
_data_pre = train_data[train_data[time_var]>int(time)].reset_index(drop=True)
_freq_post = _data_post[[id_var, event, time_var]].groupby([id_var, event]).agg({time_var: len}).reset_index()
_freq_pre = _data_pre[[id_var, event, time_var]].groupby([id_var, event]).agg({time_var: len}).reset_index()
_freq_post.columns = [id_var, 'feature_name', 'feature_value_post']
_freq_pre.columns = [id_var, 'feature_name', 'feature_value_pre']
_freq_post['feature_value_post'] = _freq_post['feature_value_post']/int(time)
_freq_pre['feature_value_pre'] = _freq_pre['feature_value_pre']/((train_data[time_var].max()) - int(time))
_normChange = pd.merge(_freq_post, _freq_pre, on=[id_var, 'feature_name'], how='outer')
_normChange.fillna(0, inplace=True)
_normChange['feature_value'] = np.where(_normChange['feature_value_post']>_normChange['feature_value_pre'], 1, 0)
_normChange.drop(['feature_value_post', 'feature_value_pre'], axis=1, inplace=True)
_normChange['feature_name'] = 'normChange__' + event + '__' + _normChange['feature_name'].astype(str) + '__' + str(time)

_normChange = _normChange.reset_index(drop=True)
_df1 = pd.DataFrame(_normChange['feature_name'].unique().tolist(), columns=['feature_name'])
_df2 = pd.DataFrame(_normChange[id_var].unique().tolist(), columns=[id_var])
_df1['key'] = 1
_df2['key'] = 1
_normTotal = pd.merge(_df2, _df1, on='key')
_normTotal.drop(['key'], axis=1, inplace=True)
_normTotal = pd.merge(_normTotal, _normChange, on=[id_var, 'feature_name'], how='left')
_normTotal.fillna(0, inplace=True)
_df3 = train_data[[id_var,y_var]].drop_duplicates().reset_index(drop=True)
_normTotal = _normTotal.merge(_df3, on=id_var, how='left')
normTotal = _normTotal.copy()

_avg1 = normTotal.loc[normTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
_avg1.columns = ['feature_name', 'avg_1']
_sd1 = normTotal.loc[normTotal[y_var]==1,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
_sd1.columns = ['feature_name', 'sd_1']
_avg0 = normTotal.loc[normTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').mean().reset_index()
_avg0.columns = ['feature_name', 'avg_0']
_sd0 = normTotal.loc[normTotal[y_var]==0,['feature_name', 'feature_value']].groupby('feature_name').agg(np.std).reset_index()
_sd0.columns = ['feature_name', 'sd_0']

_fitness_value = pd.merge(_avg1, _avg0, on='feature_name', how='left')
_fitness_value = pd.merge(_fitness_value, _sd1, on='feature_name', how='left')
_fitness_value = pd.merge(_fitness_value, _sd0, on='feature_name', how='left')

fitness = _fitness_value[_fitness_value.feature_name==feature_name]
fitness['coefficient_of_variance'] = fitness.apply(fitness_calculation, axis=1)
fitness ## Calculated Fitness for NormChange..

C:\Users\nr10863\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
10,normChange__event_name__event_10__60,0.020433,0.018123,0.141506,0.133401,1.062894


In [0]:
allFeatures[allFeatures.feature_name==feature_name] ## NormChange score from Fitness File.

,feature_name,avg_1,avg_0,sd_1,sd_0,coefficient_of_variance
28464,normChange__event_name__event_10__60,0.020433,0.018123,0.141506,0.133401,1.062894
